<a href="https://colab.research.google.com/github/roxyrong/w281-project/blob/main/feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%%capture
!pip install ipython-autotime
%load_ext autotime

time: 371 µs (started: 2023-12-11 06:46:08 +00:00)


In [ ]:
import os
import time
import copy
import gc
from glob import glob
import numpy as np
from numpy import fft
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from skimage.feature import hog, canny
from scipy.cluster.vq import kmeans, vq
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

import cv2
from PIL import Image, ImageStat

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score

time: 6.02 s (started: 2023-12-11 06:46:08 +00:00)


# Data Loading & Cleaning

In [ ]:
def load_dataset(datapath):
    df = pd.DataFrame()

    for folder in os.listdir(datapath):
      files = glob(pathname= str(datapath + folder + "/*.jpg"))
      df = pd.concat([df, pd.DataFrame( { "filename": files ,
                                          "category": folder } ) ])
    df = df.reset_index(drop=True)

    df["image"] = None
    df["gray_image"] = None
    df["hsv_image"] = None
    df["resolution"] = None
    for idx, row in df.iterrows():
      img = Image.open(row["filename"])
      df.at[idx, "image"] = np.array(img, dtype=np.float32).flatten() # (150, 150, 3)
      df.at[idx, "gray_image"] = np.array(img.convert("L")).flatten() # (150, 150)
      df.at[idx, "hsv_image"] = np.array(img.convert("HSV")).flatten() # (150, 150, 3)
      df.at[idx, "resolution"]  = img.size

    df = df[df["resolution"] == (150, 150)]

    return df


# load directly
# train_path = "drive/MyDrive/github/w281-project-me/dataset/seg_train/seg_train/"
# test_path = "drive/MyDrive/github/w281-project-me/dataset/seg_test/seg_test/"
# train_df = load_dataset(train_path)
# test_df = load_dataset(test_path)

# save to parquet locally
# train_df.to_parquet("drive/MyDrive/github/w281-project-me/dataset/train_df_3.parquet.gzip",compression="gzip")
# test_df.to_parquet("drive/MyDrive/github/w281-project-me/dataset/test_df.parquet.gzip",compression="gzip")

time: 1.44 ms (started: 2023-12-11 06:46:14 +00:00)


In [ ]:
def load_dataset_parquet(datapath):
    df = pd.read_parquet(datapath)
    for idx, row in df.iterrows():
      df.at[idx, "image"] = df.at[idx, "image"].reshape((150, 150, 3)).astype("uint8")
      df.at[idx, "gray_image"] = df.at[idx, "gray_image"].reshape((150, 150)).astype("uint8")
      df.at[idx, "hsv_image"] = df.at[idx, "hsv_image"].reshape((150, 150, 3)).astype("uint8")
    return df


def load_train(frac=1):
    train_1 = load_dataset_parquet("drive/MyDrive/github/w281-project-me/dataset/train_df_1.parquet.gzip")
    train_2 = load_dataset_parquet("drive/MyDrive/github/w281-project-me/dataset/train_df_2.parquet.gzip")
    train_3 = load_dataset_parquet("drive/MyDrive/github/w281-project-me/dataset/train_df_3.parquet.gzip")

    train_df = pd.concat([train_1, train_2, train_3], ignore_index=True)

    train_df = train_df.sample(frac=frac)
    return train_df

time: 818 µs (started: 2023-12-11 06:46:14 +00:00)


In [ ]:
# load parquet from local
test_path = "drive/MyDrive/github/w281-project-me/dataset/test_df.parquet.gzip"
train_df = load_train()
test_df = load_dataset_parquet(test_path)
gc.collect()

0

time: 55.3 s (started: 2023-12-11 06:46:14 +00:00)


In [ ]:
print(train_df.groupby("category")["image"].count())
print(test_df.groupby("category")["image"].count())

category
buildings    2190
forest       2263
glacier      2387
mountain     2495
sea          2270
street       2381
Name: image, dtype: int64
category
buildings    437
forest       473
glacier      549
mountain     523
sea          510
street       501
Name: image, dtype: int64
time: 7.48 ms (started: 2023-12-11 06:47:10 +00:00)


# Feature Engineering

In [ ]:
## RGB feature
def add_rgb_feature(row):
    r, g, b = np.mean(train_df.iloc[0]["image"], axis=(0, 1))
    row["mean_r"] = r
    row["mean_g"] = g
    row["mean_b"] = b

    return row

time: 464 µs (started: 2023-12-11 06:47:10 +00:00)


In [ ]:
## HSV feature
def add_hsv_feature(row):
    image = Image.fromarray(row["hsv_image"].astype("uint8"))
    hsv_stats = ImageStat.Stat(image)

    row["mean_h"] = hsv_stats.mean[0]
    row["mean_s"] = hsv_stats.mean[1]
    row["mean_v"] = hsv_stats.mean[2]

    row["stddev_h"] = hsv_stats.stddev[0]
    row["stddev_s"] = hsv_stats.stddev[1]
    row["stddev_v"] = hsv_stats.stddev[2]

    row["hsv_histogram"] = np.array(image.histogram())

    return row

time: 566 µs (started: 2023-12-11 06:47:10 +00:00)


In [ ]:
## HOG feature
def add_hog_feature(row):
  row["hog_feature"], row["hog_image"] = hog(row["image"],
                                             orientations=8,
                                             pixels_per_cell=(15, 15),
                                             cells_per_block=(1, 1),
                                             visualize=True,
                                             channel_axis=-1)

  return row

time: 484 µs (started: 2023-12-11 06:47:10 +00:00)


In [ ]:
## Canny Edge Detection
def add_canny_feature(row):
  row["canny_feature"] = canny(row["gray_image"], sigma=2).flatten()
  return row

time: 545 µs (started: 2023-12-11 06:47:10 +00:00)


In [ ]:
def add_fourier_feature(row):
  image = row["gray_image"]
  ydim = image.shape[0]
  xdim = image.shape[1]
  win = np.outer(np.hanning(ydim), np.hanning(xdim))
  win = win/np.mean(win)
  fourier = fft.fftshift(fft.fft2(image * win))
  Fmag = np.abs(fourier)
  row["fourier_feature"] = np.log(Fmag).flatten()
  return row

time: 571 µs (started: 2023-12-11 06:47:10 +00:00)


In [ ]:
## HED Edge Detection
## https://github.com/s9xie/hed

# def blob_batch_processing(original_images):
#   images = copy.deepcopy(original_images)
#   images = images.apply(lambda x: cv2.resize(x, (299, 299)))
#   images = np.array(images)
#   blob = cv2.dnn.blobFromImages(images, scalefactor=1, size=(299, 299),
#     mean=(105, 117, 123), #MODEL MEAN VALUE
#     swapRB=True, crop=False)
#   blob = list(blob)
#   blob = [b[np.newaxis, :]for b in blob]
#   return blob

# # The pre-trained model that OpenCV uses has been trained in Caffe framework
# proto_path = "drive/MyDrive/github/w281-project-me/third_party/HED/deploy.prototxt"
# model_path = "drive/MyDrive/github/w281-project-me/third_party/HED/hed_pretrained_bsds.caffemodel"
# net = cv2.dnn.readNetFromCaffe(proto_path, model_path)

# def add_HED_feature(row):
#   net.setInput(row["blob"])
#   hed = net.forward()
#   hed = cv2.resize(np.squeeze(hed), (150, 150))
#   hed = cv2.cvtColor(hed, cv2.COLOR_GRAY2BGR)
#   hed = (255 * hed).astype("uint8")
#   row["hed"] = hed.flatten()
#   return row


time: 234 µs (started: 2023-12-11 06:47:10 +00:00)


In [ ]:
def add_bag_of_visual_word_feature(train_df, test_df):
    extractor = cv2.xfeatures2d.SIFT_create()
    k = 200 # number of visual words

    def add_sift(row):
        img = row["gray_image"]
        img_keypoints, img_descriptors = extractor.detectAndCompute(img, None)
        row["keypoint"] = img_keypoints
        row["descriptor"] = img_descriptors
        return row

    def add_visual_words(row):
        img_descriptors = row["descriptor"]
        img_visual_words = []
        if img_descriptors is not None:
          img_visual_words, distance = vq(img_descriptors, codebook)
          row["visual_word"] = img_visual_words

        img_frequency_vector = np.zeros(k)
        for word in img_visual_words:
            img_frequency_vector[word] += 1
        row["frequency_vector"] = img_frequency_vector
        return row

    # creating codebook for visual words
    train_df = train_df.apply(add_sift, axis=1)
    test_df = test_df.apply(add_sift, axis=1)

    sample_index = train_df[train_df["descriptor"].notna()].sample(frac=0.2).index
    all_descriptors = np.vstack(train_df.loc[sample_index]["descriptor"].to_numpy())
    codebook, variance = kmeans(all_descriptors, k, 1)

    # compute tfidf
    train_df = train_df.apply(add_visual_words, axis=1)
    test_df = test_df.apply(add_visual_words, axis=1)

    train_frequency_vectors = np.vstack(train_df["frequency_vector"])
    test_frequency_vectors = np.vstack(test_df["frequency_vector"])
    idf = np.log(len(train_df) / np.sum(train_frequency_vectors > 0, axis=0))
    train_tfidf =  train_frequency_vectors * idf
    test_tfidf = test_frequency_vectors * idf
    train_df["bag_of_visual_words"] = list(train_tfidf)
    test_df["bag_of_visual_words"] = list(test_tfidf)
    return train_df, test_df

time: 1.09 ms (started: 2023-12-11 06:47:10 +00:00)


In [ ]:
#RGB
train_df = train_df.apply(add_rgb_feature, axis=1) # 52 sec
test_df = test_df.apply(add_rgb_feature, axis=1) # 13 sec

gc.collect()

0

time: 40.9 s (started: 2023-12-11 06:47:10 +00:00)


In [ ]:
#HSV (768, 0)
train_df = train_df.apply(add_hsv_feature, axis=1) # 52 sec
test_df = test_df.apply(add_hsv_feature, axis=1) # 13 sec

#filter out black & white images
train_df = train_df[train_df["mean_h"] > 5]
test_df = test_df[test_df["mean_h"] > 5]

gc.collect()

0

time: 1min 16s (started: 2023-12-11 06:47:51 +00:00)


In [ ]:
#VGG16 (8192, 0)
vgg_model = VGG16(include_top=False, input_shape=(150, 150, 3))

train_df["vgg_input"] = train_df["image"].apply(preprocess_input)
train_df["vgg16_feature"] = list(vgg_model.predict(np.stack(train_df["vgg_input"].to_numpy()),
                                                 verbose=False).reshape(len(train_df), -1))

train_df.drop(columns=['vgg_input'], inplace=True)
gc.collect()

test_df["vgg_input"] = test_df["image"].apply(preprocess_input)
test_df["vgg16_feature"] = list(vgg_model.predict(np.stack(test_df["vgg_input"].to_numpy()),
                                                 verbose=False).reshape(len(test_df), -1))
test_df.drop(columns=['vgg_input'], inplace=True)

gc.collect()

643

time: 2min 52s (started: 2023-12-11 07:03:21 +00:00)


In [ ]:
#HOG: shape = (800, 0)
train_df = train_df.apply(add_hog_feature, axis=1)  # 211 sec
test_df = test_df.apply(add_hog_feature, axis=1) # 44 sec

gc.collect()

0

time: 5min 6s (started: 2023-12-11 07:21:03 +00:00)


In [ ]:
#Canny: shape = (150, 150) Not using due to low performance
train_df = train_df.apply(add_canny_feature, axis=1) # 64 sec
test_df = test_df.apply(add_canny_feature, axis=1) # 13 sec

gc.collect()

0

time: 1min 47s (started: 2023-12-11 07:26:10 +00:00)


In [ ]:
#HED shape = (150, 150, 3) Not using due to long runtime
# train_df["blob"] = blob_batch_processing(train_df["image"])
# train_df = train_df.apply(add_HED_feature, axis=1)
# test_df = test_df.apply(add_HED_feature, axis=1)

time: 241 µs (started: 2023-12-11 07:27:58 +00:00)


In [ ]:
#fourier: shape = (150, 150)
train_df = train_df.apply(add_fourier_feature, axis=1)
test_df = test_df.apply(add_fourier_feature, axis=1)

gc.collect()

0

time: 38.8 s (started: 2023-12-11 07:27:58 +00:00)


In [ ]:
# bag of visual words
train_df, test_df = add_bag_of_visual_word_feature(train_df, test_df)

gc.collect()

0

time: 4min 37s (started: 2023-12-11 07:28:37 +00:00)


# Feature Selection

In [ ]:
def pca_analysis(df, feature_dimensions):
    explained_variance_dict = {}

    for feature, num_component in feature_dimensions.items():
        print(feature)
        feature_vector = np.stack(df[feature].to_numpy())

        explained_variance = []
        max_components = min(num_component, len(df))

        for components in range(5, max_components + 1, 5):
            pca = PCA(n_components=components)
            pca.fit(feature_vector)
            total_variance = np.sum(pca.explained_variance_ratio_)
            explained_variance.append(total_variance)

        explained_variance_dict[feature] = explained_variance

    return explained_variance_dict


def pca_analysis_plot(explained_variance_dict):
    plt.figure(figsize=(10, 6))

    for feature, variances in explained_variance_dict.items():
        plt.plot([i * 5 for i in range(1, len(variances) + 1)],
                 variances,
                 label=feature)

    plt.title("Explained Variance by Feature from PCA Analysis")
    plt.xlabel("Component Number")
    plt.ylabel("Explained Variance")
    plt.legend()
    plt.show()

time: 1.2 ms (started: 2023-12-11 07:33:14 +00:00)


In [ ]:
feature_dimensions = {
    "hsv_histogram" : 768,
    "hog_feature": 800,
    "canny_feature": 22500,
    "vgg16_feature": 8192,
    "fourier_feature": 22500,
    "bag_of_visual_words": 200
}

# explained_variance_dict = pca_analysis(train_df.sample(700),
                                      #  feature_dimensions)

# pca_analysis_plot(explained_variance_dict)

time: 409 µs (started: 2023-12-11 07:33:14 +00:00)


In [ ]:
feature_pca_components = {
    "hsv_histogram" : 50,
    "hog_feature": 100,
    "vgg16_feature": 200,
    "fourier_feature": 25,
    "bag_of_visual_words": 40
}

pca_features = [f"pca_{feature}" for feature in feature_pca_components.keys()]
print(pca_features)

['pca_hsv_histogram', 'pca_hog_feature', 'pca_vgg16_feature', 'pca_fourier_feature', 'pca_bag_of_visual_words']
time: 620 µs (started: 2023-12-11 07:33:14 +00:00)


In [ ]:
def apply_pca(train_df, test_df, feature_pca_components):
    for feature, num_component in feature_pca_components.items():
        pca = PCA(n_components=num_component)
        pca.fit(np.stack(train_df[feature].to_numpy()))
        train_df[f"pca_{feature}"] = list(pca.transform(np.stack(train_df[feature].to_numpy())))
        print(feature, np.sum(pca.explained_variance_ratio_))
        test_df[f"pca_{feature}"] = list(pca.transform(np.stack(test_df[feature].to_numpy())))
    return train_df, test_df

train_df, test_df = apply_pca(train_df, test_df, feature_pca_components)

gc.collect()

hsv_histogram 0.8734238434244337
hog_feature 0.6916298380552427
vgg16_feature 0.570367
fourier_feature 0.4756601873218522
bag_of_visual_words 0.7636443479635329


0

time: 37.9 s (started: 2023-12-11 07:33:14 +00:00)


In [ ]:
train_features_path = "drive/MyDrive/github/w281-project-me/dataset/train_features_full.parquet.gzip"
test_features_path = "drive/MyDrive/github/w281-project-me/dataset/test_features_full.parquet.gzip"

features = ["mean_r", "mean_g", "mean_b", "mean_h", "mean_s", "mean_v", "stddev_h", "stddev_s", "stddev_v",
            "pca_hsv_histogram", "pca_hog_feature", "pca_vgg16_feature",
            "pca_fourier_feature", "pca_bag_of_visual_words"]

train_features_df = train_df[features + ["category", "image"]].copy()
for idx, row in train_features_df.iterrows():
  train_features_df.at[idx, "image"] = train_features_df.at[idx, "image"].flatten()
train_features_df.to_parquet(train_features_path, compression="gzip")

test_features_df = test_df[features + ["category", "image"]]
for idx, row in test_features_df.iterrows():
    test_features_df.at[idx, "image"] = test_features_df.at[idx, "image"].flatten()
test_features_df.to_parquet(test_features_path, compression="gzip")

time: 1min 35s (started: 2023-12-11 07:37:05 +00:00)
